<a href="https://colab.research.google.com/github/shaikasheesh/CapstoneGL/blob/master/Model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
data_final = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_final.csv')
data_final.head(2)

,Unnamed: 0,Bankrupt?,ROA(C) before interest and depreciation before interest,Operating Gross Margin,Operating Profit Rate,Non-industry income and expenditure/revenue,Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,...,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Equity to Long-term Liability,Cash Flow to Total Assets,Total assets to GNP price,No-credit Interval,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT)
0,0,1.0,0.370594,0.601457,0.998969,0.302646,-3.685025e-14,-0.173178,0.458143,1.266458,...,0.593831,-0.172398,0.671568,0.801932,0.126549,0.637555,1.655335,0.622879,0.026601,0.564050
1,1,1.0,0.464291,0.610235,0.998946,0.303556,3.466254e-15,-0.173178,0.461867,1.021944,...,0.593916,0.287330,0.671570,0.962361,0.120916,0.641100,1.446289,0.623652,0.026722,0.570175


In [4]:
x = data_final.drop('Bankrupt?',axis =1)
y = data_final['Bankrupt?']

In [5]:
from sklearn.model_selection import train_test_splitbb

In [6]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.7,stratify= y)

In [7]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,classification_report

In [8]:
def model(func,xtrain,ytrain,xtest,ytest,flag):
    mdl = func.fit(xtrain,ytrain)
    ypred_train = mdl.predict(xtrain)
    ypred_test = mdl.predict(xtest)
    if flag == 'X':
        return accuracy_score(ytrain,ypred_train),recall_score(ytrain,ypred_train),precision_score(ytrain,ypred_train),f1_score(ytrain,ypred_train)
    else:
        return accuracy_score(ytest,ypred_test),recall_score(ytest,ypred_test),precision_score(ytest,ypred_test),f1_score(ytest,ypred_test)
    

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
lr = LogisticRegression()

In [56]:
df_log_reg_scorecard = pd.DataFrame(columns=['model','Train accuracy','Test accuracy',
                                             'Train recall','Test recall',
                                             'Train precision','Test precision',
                                             'Train f1score','Test f1score'])
def update_test_score(model,train_acc,test_acc,train_recall,test_recall,train_preci,test_preci,train_f1,test_f1):
    global df_log_reg_scorecard
    df_log_reg_scorecard = df_log_reg_scorecard.append({'model':model,
                                'Train accuracy':train_acc,
                                'Test accuracy':test_acc,
                                'Train recall':train_recall,                       
                                'Test recall':test_recall,
                                'Train precision':train_preci,
                                'Test precision':test_preci,
                                'Train f1score':train_f1 ,                      
                                'Test f1score':test_f1},ignore_index = True)

In [57]:
train_acc,train_recall,train_preci,train_f1 = model(lr,xtrain,ytrain,xtest,ytest,'X')
test_acc,test_recall,test_preci,test_f1 = model(lr,xtrain,ytrain,xtest,ytest,'')
update_test_score('Logistic without tuning',train_acc,test_acc,train_recall,test_recall,train_preci,test_preci,train_f1,test_f1)

In [13]:
df_log_reg_scorecard

,model,Train accuracy,Test accuracy,Train recall,Test recall,Train precision,Test precision,Train f1score,Test f1score
0,Logistic without tuning,0.973944,0.973953,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#hyper parameter tuning
#parameters considered for log are
#solver,penalty,C,max_iter
#ref:https://medium.com/codex/do-i-need-to-tune-logistic-regression-hyperparameters-1cb2b81fca69
parameter = [{'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'penalty':['l2'],
              'C': [100, 10, 1.0, 0.1, 0.01],
              'max_iter':[10,30,50,75,100]
             }]
gs = GridSearchCV(lr,param_grid=parameter,cv=5,scoring='recall')
mdl = gs.fit(xtrain,ytrain)

In [18]:
mdl.best_params_

{'C': 100, 'max_iter': 10, 'penalty': 'l2', 'solver': 'sag'}

In [17]:
from warnings import filterwarnings
filterwarnings('ignore')

In [19]:
parameter = [{'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'penalty':['l2'],
              'C': [100,120,110,150],
              'max_iter':[2,5,7,10]
             }]
gs = GridSearchCV(lr,param_grid=parameter,cv=5,scoring='recall')
mdl_1 = gs.fit(xtrain,ytrain)

In [20]:
mdl_1.best_params_

{'C': 100, 'max_iter': 2, 'penalty': 'l2', 'solver': 'newton-cg'}

In [24]:
parameter = [{'solver':['liblinear'],
              'penalty':['l1','l2'],
              'C': [100,120,110,150],
              'max_iter':[2,5,7,10]
             }]
gs = GridSearchCV(lr,param_grid=parameter,cv=5,scoring='recall')
mdl_2 = gs.fit(xtrain,ytrain)

In [25]:
mdl_2.best_params_

{'C': 100, 'max_iter': 2, 'penalty': 'l2', 'solver': 'liblinear'}

In [58]:
#{'C': 100, 'max_iter': 10, 'penalty': 'l2', 'solver': 'sag'}
log_tuned_mdl_1 = LogisticRegression(penalty='l2',max_iter=10,C = 100,solver = 'sag')
train_acc,train_recall,train_preci,train_f1 = model(log_tuned_mdl_1,xtrain,ytrain,xtest,ytest,'X')
test_acc,test_recall,test_preci,test_f1 = model(log_tuned_mdl_1,xtrain,ytrain,xtest,ytest,'')
update_test_score("'Logistic with tuning model_1 with penalty='l2',max_iter=10,C = 100,solver = 'sag'",train_acc,test_acc,train_recall,test_recall,train_preci,test_preci,train_f1,test_f1)

In [59]:
#{'C': 100, 'max_iter': 2, 'penalty': 'l2', 'solver': 'newton-cg'}
log_tuned_mdl_2 = LogisticRegression(penalty='l2',max_iter=2,C = 100,solver = 'newton-cg')
train_acc,train_recall,train_preci,train_f1 = model(log_tuned_mdl_2,xtrain,ytrain,xtest,ytest,'X')
test_acc,test_recall,test_preci,test_f1 = model(log_tuned_mdl_2,xtrain,ytrain,xtest,ytest,'')
update_test_score("'Logistic with tuning model_2 with penalty='l2',max_iter=2,C = 100,solver = 'newton-cg'",train_acc,test_acc,train_recall,test_recall,train_preci,test_preci,train_f1,test_f1)

In [60]:
#{'C': 100, 'max_iter': 2, 'penalty': 'l2', 'solver': 'liblinear'}
log_tuned_mdl_3 = LogisticRegression(penalty='l2',max_iter=2,C = 100,solver = 'liblinear')
train_acc,train_recall,train_preci,train_f1 = model(log_tuned_mdl_3,xtrain,ytrain,xtest,ytest,'X')
test_acc,test_recall,test_preci,test_f1 = model(log_tuned_mdl_3,xtrain,ytrain,xtest,ytest,'')
update_test_score("'Logistic with tuning model_3 with 'C': 100, 'max_iter': 2, 'penalty': 'l2', 'solver': 'liblinear'",train_acc,test_acc,train_recall,test_recall,train_preci,test_preci,train_f1,test_f1)

In [52]:
#saga’ - [‘elasticnet’, ‘l1’, ‘l2’, ‘none’]
parameter = [{'solver':['saga'],
              'penalty':['l1','l2','elasticnet','none'],
              'C': [100,120,110,150],
              'max_iter':[2,5,7,10]
             }]
gs = GridSearchCV(lr,param_grid=parameter,cv=5,scoring='recall')
mdl_3 = gs.fit(xtrain,ytrain)

In [53]:
mdl_3.best_params_

{'C': 100, 'max_iter': 2, 'penalty': 'l1', 'solver': 'saga'}

In [61]:
#{'C': 100, 'max_iter': 2, 'penalty': 'l1', 'solver': 'saga'}
log_tuned_mdl_4 = LogisticRegression(penalty='l1',max_iter=2,C = 100,solver = 'saga')
train_acc,train_recall,train_preci,train_f1 = model(log_tuned_mdl_4,xtrain,ytrain,xtest,ytest,'X')
test_acc,test_recall,test_preci,test_f1 = model(log_tuned_mdl_4,xtrain,ytrain,xtest,ytest,'')
update_test_score("'Logistic with tuning model_4 with penalty='l1',max_iter=2,C = 100,solver = 'saga'",train_acc,test_acc,train_recall,test_recall,train_preci,test_preci,train_f1,test_f1)

In [62]:
df_log_reg_scorecard

,model,Train accuracy,Test accuracy,Train recall,Test recall,Train precision,Test precision,Train f1score,Test f1score
0,Logistic without tuning,0.973944,0.973953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,'Logistic with tuning model_1 with penalty='l2...,0.660828,0.651685,0.134454,0.117647,0.010944,0.009331,0.020240,0.017291
2,'Logistic with tuning model_2 with penalty='l2...,0.594701,0.591931,0.210084,0.235294,0.014029,0.015544,0.026302,0.029162
3,"'Logistic with tuning model_3 with 'C': 100, '...",0.621852,0.621553,0.193277,0.235294,0.013906,0.016807,0.025945,0.031373
4,'Logistic with tuning model_4 with penalty='l1...,0.658857,0.646578,0.142857,0.176471,0.011541,0.013657,0.021357,0.025352
